In [ ]:
data_dir "../data/"
gbfile=data_dir+'GCF_002163715.1_ASM216371v1_genomic.gbff'

In [ ]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
import pandas as pd

In [ ]:
'''parse_genbank_file_for_utr

'''
def parse_genbank_file_for_utr(gbfile:str,basepairs=200, end_basepairs=180,start_basepairs=60, filter_facing_seqs=True):

    records = []
    for record in SeqIO.parse(gbfile,'genbank'):
        records.append(record)

    print("[+] Working on {} genbank records".format(len(records)))
    print("[+] Extracting all sequences with location from end plus {}bp".format(basepairs))
    count = 0
    feature_dict = {}
    #list inherits all protein identifier from start to end of gb file
    ordered_keys = []
    for index, rec in enumerate(records):
        try:
            for feature in rec.features:
                count += 1
                try:
                    if feature.type == 'gene':
                        #if condition to get all sequences, even the ones without any new annotation
                        if 'old_locus_tag' in feature.qualifiers:
                            if feature.location.strand == -1:
                                seq = records[0].seq[feature.location.end.position:feature.location.end.position+basepairs].reverse_complement()
                                loc = (feature.location.end.position,feature.location.end.position+basepairs)

                            elif feature.location.strand == 1:
                                seq = records[0].seq[feature.location.start.position-basepairs:feature.location.start.position]
                                loc = (feature.location.start.position-basepairs,feature.location.start.position)
                            
                            feature_dict[feature.qualifiers['old_locus_tag'][0]] = [feature.location,loc,seq,index]
                            ordered_keys.append(feature.qualifiers['old_locus_tag'][0])


                        else:
                            if feature.location.strand == -1:
                                seq = records[0].seq[feature.location.end.position:feature.location.end.position+basepairs].reverse_complement()
                                loc = (feature.location.end.position,feature.location.end.position+basepairs)
                            elif feature.location.strand == 1:
                                seq = records[0].seq[feature.location.start.position-basepairs:feature.location.start.position]
                                loc = (feature.location.start.position-basepairs,feature.location.start.position)
                            
                            feature_dict[feature.qualifiers['locus_tag'][0]] = [feature.location,loc,seq,index]
                            ordered_keys.append(feature.qualifiers['locus_tag'][0])

                except Exception as e:
                    print("[-] ERROR: {}".format(e))
                    pass
        except Exception as e:
            print("[-] ERROR: {}".format(e))
            continue
    
    #parsing feature dict - evaluating end location + basepairs of target sequences 
    target_promotor_seqs = {}
    print("[+] Try Parsing Results For Putative Promotor Sequences")
    print("\t[+] Applied filters: sequences have to be bigger than {} and smaller than {}".format(end_basepairs,start_basepairs))
    if filter_facing_seqs is True:
        print("\t[+] Filter out end facing sequences (-1 1 | 1 -1) that might share putative promotor sequences")
    for index, key in enumerate(ordered_keys):
        #do not evaluate start and end sequences
        if index > 0 and index < len(ordered_keys)-1:
            cds = feature_dict[key]
            
            if cds[0].strand == -1:#if the strand is negative look at the next sequence 
                key_before = ordered_keys[index+1]
                start = cds[0].end.position                
                cds_before = feature_dict[key_before]
                start_next_gene = cds_before[0].start.position
        

            elif cds[0].strand == 1:#if the strand is positive look at the sequence before
                key_before = ordered_keys[index-1]
                start = cds[0].start.position
                cds_before = feature_dict[key_before]
                start_next_gene = cds_before[0].end.position
            
            #filter out sequences that "look on each other"
            if filter_facing_seqs is True:
                if((cds[0].strand == -1 and cds_before[0].strand == 1) or (cds[0].strand == 1 and cds_before[0].strand == -1)) == False:
                    if cds[0].strand == -1:
                        if(abs(start_next_gene - start) <= end_basepairs) and (abs(start_next_gene - start) >= start_basepairs) :

                            if (int(cds[0].end.position) < int(cds_before[0].start.position)):
                                print("\t[+] ",key, key_before, abs(start_next_gene - start), cds[0].strand, cds_before[0].strand)
                                target_promotor_seqs[key] = [start_next_gene, start]
                            else:
                                print("\t\t[+] unusual overlapping sequences: {} - {} - strand: {}".format(key,key_before, cds[0].strand))

                    elif cds[0].strand == 1:
                        if(abs(start_next_gene - start) <= end_basepairs) and (abs(start_next_gene - start) >= start_basepairs):
                            if(int(cds_before[0].end.position) < int(cds[0].start.position)):
                                print("\t[+] ",key, key_before, abs(start_next_gene - start), cds[0].strand, cds_before[0].strand)
                                target_promotor_seqs[key] = [start_next_gene, start]
                            else:
                                print("\t\t[+] unusual overlapping sequences: {} - {} - strand: {}".format(key,key_before, cds[0].strand))


            else:
                if cds[0].strand == -1:
                    if (abs(start_next_gene - start) <= end_basepairs) and (abs(start_next_gene - start) >= start_basepairs) :
                        if(int(cds[0].end.position) < int(cds_before[0].start.position)) :
                            print("\t[+] ",key, key_before, abs(start_next_gene - start), cds[0].strand, cds_before[0].strand)
                            target_promotor_seqs[key] = [start_next_gene, start]
                        else:
                            print("\t\t[+] unusual overlapping sequences: {} - {} - strand: {}".format(key,key_before, cds[0].strand))                                       

                elif cds[0].strand == 1:
                    if(abs(start_next_gene - start) <= end_basepairs) and (abs(start_next_gene - start) >= start_basepairs):
                        if (int(cds_before[0].end.position) < int(cds[0].start.position)):
                            print("\t[+] ",key, key_before, abs(start_next_gene - start), cds[0].strand, cds_before[0].strand)
                            target_promotor_seqs[key] = [start_next_gene, start]
                        else:
                            print("\t\t[+] unusual overlapping sequences: {} - {} - strand: {}".format(key,key_before, cds[0].strand))

                            
        targets = target_promotor_seqs.keys()
        seqs = {}
        for rec in records:
            try:
                for feature in rec.features:
                    count += 1
                    try:
                        if feature.type == 'gene':
                            if 'old_locus_tag' in feature.qualifiers:
                                if feature.qualifiers['old_locus_tag'][0] in targets:
                                    if feature.location.strand == -1:
                                        seqs[feature.qualifiers['old_locus_tag'][0]] = rec.seq[feature.location.end.position:feature.location.end.position+basepairs]
                                    else:
                                        seqs[feature.qualifiers['old_locus_tag'][0]] = rec.seq[feature.location.start.position-basepairs:feature.location.start.position]

                            else:
                                if feature.qualifiers['locus_tag'][0] in targets:
                                    if feature.location.strand == -1:
                                        seqs[feature.qualifiers['locus_tag'][0]] = rec.seq[feature.location.end.position:feature.location.end.position+basepairs]
                                    else:
                                        seqs[feature.qualifiers['locus_tag'][0]] = rec.seq[feature.location.start.position-basepairs:feature.location.start.position]

                                
                    except Exception as e:
                        print("[-] ERROR: {}".format(e))
                        pass
            except Exception as e:
                print("[-] ERROR: {}".format(e))
                continue        

            
    print("[+] DONE")
    return feature_dict, records, target_promotor_seqs, ordered_keys, seqs

In [ ]:
'''extract_sequences_based_on_target_promotor_seqs_dict

'''
def extract_sequences_based_on_target_promotor_seqs_dict(feature_dict:dict,target_promotor_seqs:dict, records:list)->dict:
    seqs = {}
    
    print("[+] Trying to extract putative promotor sequences")
    for target in target_promotor_seqs.keys():
        if feature_dict[target][0].strand == 1:
            loc_cds_start = target_promotor_seqs[target][0]
            loc_cds_end = target_promotor_seqs[target][1]
            try:
                seqs[target] = records[feature_dict[target][3]].seq[loc_cds_start:loc_cds_end].__str__()
            except Exception as e:
                print("[-] ERROR: {}".format(e))
        if feature_dict[target][0].strand == -1:
            loc_cds_start = target_promotor_seqs[target][1]
            loc_cds_end = target_promotor_seqs[target][0]
            try:
                seqs[target] = records[feature_dict[target][3]].seq[loc_cds_start:loc_cds_end].reverse_complement().__str__()
            except Exception as e:
                print("[-] ERROR: {}".format(e))
    print("[+] DONE")
    return seqs

In [ ]:
#19.09.2022 downloaded from NCBI refseq ftp fileserver
res = parse_genbank_file_for_utr(gbfile, end_basepairs=170, start_basepairs=60)
seqs = extract_sequences_based_on_target_promotor_seqs_dict(res[0],res[2], res[1])

In [ ]:
print("[+] Number of target promotor sequences: {}".format(len(res[2])))

In [ ]:
print("[+] Number of extracted sequences: {}".format(len(seqs)))

In [ ]:
df_seqs=pd.DataFrame(seqs,index=['seq']).transpose()
df_seqs = df_seqs.reset_index()
df_seqs.columns=['aep','seq']
df_seqs.head()

In [ ]:
deseq2_excel=data_dir + "excel_sheet_ordered_degs_curvibacter_aep_to_wp.xlsx"
deseq2_df=pd.read_excel(deseq2_excel)
deseq2_df.head()

In [ ]:
promotor_stength_df=deseq2_df.merge(df_seqs,on='aep')
f = lambda s: s.replace(",",".")
promotor_stength_df['Wp_Number'] = promotor_stength_df['Wp_Number'].apply(f)

In [ ]:
print("[+] Number of target promotors after merging with transcriptome dataframe: {}".format(len(promotor_stength_df)))

In [ ]:
print("[*] Extracting sequences that do not reside in the merged dataframe ...")
print("\t[*] Those sequences are mainly composed of tRNAs or other small nucleotide sequences")
diff = [d for d in list(df_seqs.aep.values) if d not in list(promotor_stength_df.aep)] # list of sequences that have putative promotor utrs but do not reside in the transcriptome dataframe
print("\t[*] DONE")
#e.g. AEP_00144 is a tRNA

In [ ]:
diff

In [ ]:
raw_reads_path=data_dir + "raw_read_counts.csv"
raw_reads_df=pd.read_csv(raw_reads_path)
raw_reads_df.columns=["Wp_Number","G1","G2","G3","Hydra1","Hydra2","Hydra3"]
raw_reads_df.head()

In [ ]:
promotor_stength_raw_df=promotor_stength_df.merge(raw_reads_df,on="Wp_Number")
promotor_stength_raw_df['readCountMeanGSamples'] = promotor_stength_raw_df[['G1','G2','G3']].mean(axis=1)
promotor_stength_raw_df['readCountMeanNormalized'] = promotor_stength_raw_df['readCountMeanGSamples']/promotor_stength_raw_df['readCountMeanGSamples'].max()
promotor_stength_raw_df.head()

In [ ]:
print("[+] Length of dataframe after merging raw read data: {}".format(len(promotor_stength_raw_df)))

In [ ]:
df_targets = promotor_stength_raw_df[['Wp_Number','aep','readCountMeanGSamples','readCountMeanNormalized','log2FoldChange','seq']]

In [ ]:
df_targets.head()

In [ ]:
#top 400 expressed regions
high_400 = df_targets.sort_values(by='readCountMeanNormalized', ascending=False)[:400]
#smallest 100 of the remaining regions
other_seqs = df_targets.sort_values(by='readCountMeanNormalized', ascending=False)[400:]

In [ ]:
high_400.head()

In [ ]:
#creating sequence length column for the subset of the smallest 100 of remaining regions
seq_length=lambda seq: len(seq)
other_seqs['seq_length']=other_seqs['seq'].apply(seq_length)
high_400['seq_length']=high_400['seq'].apply(seq_length)

In [ ]:
low_100=other_seqs.sort_values(by='seq_length',ascending=True)[:100]

In [ ]:
result_df=pd.concat([high_400,low_100])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#values for plotting
ordered_gene_list = res[3] #ordered_dict
result_gene_list = list(result_df['aep'])
steps = 1/len(ordered_gene_list)

x_values=np.arange(1,len(ordered_gene_list)+1)*steps
y_values=np.repeat(1,len(x_values))
#print(len(x_values) == len(y_values))
x2_values=[]
for key,value in zip(ordered_gene_list,x_values):
    if key in result_gene_list:
        x2_values.append(value)
y2_values=list(np.repeat(1,len(x2_values)))
#print(len(x2_values) == 500)
#print(len(x2_values) == len(y2_values))

In [ ]:
plt.figure(figsize=(30,8))
plt.scatter(x=x_values,y=y_values, s=1000)
plt.scatter(x=x2_values,y=y2_values, s=5, c='r')
plt.ylim(0.9,1.1)
plt.grid()

# Trimming Length Of Sequences

In [ ]:
#import the list of promoter sequences in 5' to 3' direction in a list
raw_library = []

# determine maximum length of promoter site to be used for expression and the maximum size of oligos to order
promoter_max_len = 98
synthesis_maxlen = 150
# determine sequences for cloning to be attached to the promoters
upstream_cloningsite = "tcgagtacgacttcggtctcaGGAGc"
downstream_cloningsite = "cAATGtgagaccgaacgtcagtgatc"
# random sequence to fill synthesis order, will be cut during cloning and has no impact on construct
fillup = "ATCGATCGCTAGCTAGCTAGCATCGACTATCGTCGATCGATCGATGCATGCATCTGTACGATCGACTAGCTAGTCGACTATCGACTGACTGACTGACTG"
finalorder = []
for i in range(len(raw_library)):
    finalorder.append(i)


for i in raw_library:
    #save the original sequence to get raw_library.index in the end
    rawstring = i
    # cut off first base to match final RBS distance in construct
    i = i[:-1]
    # trimming too long promoters
    if len(i) > promoter_max_len:
        overlength = len(i) - promoter_max_len
        i = i[overlength:]
        # adding cloning sites to both ends
    i = upstream_cloningsite + i
    i += downstream_cloningsite
    #fill up synthesis for order of equal length oligos
    i = fillup + i
    overlength_syn = len(i) - synthesis_maxlen
    i = i[overlength_syn:]
    #put new strings into final order list
    finalorder[raw_library.index(rawstring)] = i


# checks if size of the list, size of each oligo is correct
for i in finalorder:
    if len(i) == synthesis_maxlen:
        print(i)
        print("length:" + str(len(i)))
    else:
        print("ERROR: oligo has not the correct length")
if len(finalorder) == len(raw_library):
    print("size of order is equal to library:" + str(len(finalorder)))
else:
    print("ERROR: size of order doesnt match original library")

In [ ]:
#import the list of promoter sequences in 5' to 3' direction in a list
raw_library = list(result_df['seq'])

# determine maximum length of promoter site to be used for expression and the maximum size of oligos to order
promoter_max_len = 98
synthesis_maxlen = 150
# determine sequences for cloning to be attached to the promoters
# designed in snapgene by maurice mager
upstream_cloningsite = "tcgagtacgacttcggtctcaGGAGc"
downstream_cloningsite = "cAATGtgagaccgaacgtcagtgatc"

# random sequence to fill synthesis order, will be cut during cloning and has no impact on construct
# filling of random nucleotides behind cloning sites if sequence too small e.g. < 98
fillup = "ATCGATCGCTAGCTAGCTAGCATCGACTATCGTCGATCGATCGATGCATGCATCTGTACGATCGACTAGCTAGTCGACTATCGACTGACTGACTGACTG"

aep_seq={}
for aep,seq in zip(result_df['aep'],result_df['seq']):
    #save the original sequence to get raw_library.index in the end
    rawstring = seq
    # cut off first base to match final RBS distance in construct - why?
    seq = seq[:-1]
    # trimming too long promoters
    if len(seq) > promoter_max_len:
        print("[+] Trimming sequence: {} with length: {}".format(aep,len(seq)))
        overlength = len(seq) - promoter_max_len
        seq = seq[overlength:]
        
    # adding cloning sites to both ends
    seq = upstream_cloningsite + seq
    seq += downstream_cloningsite
    
    #fill up synthesis for order of equal length oligos
    seq = fillup + seq
    overlength_syn = len(seq) - synthesis_maxlen
    seq = seq[overlength_syn:]

    aep_seq[aep]=seq


# checks if size of the list, size of each oligo is correct
for aep in aep_seq.keys():
    if len(aep_seq[aep]) == synthesis_maxlen:
        print("[+] {}".format(aep))
        print("\t[+] length:" + str(len(aep_seq[aep])))
    else:
        print("[-] ERROR: oligo for {} has not the correct length".format(aep))
if len(aep_seq.keys()) == len(raw_library):
    print("[+] size of order is equal to library")
else:
    print("[-] ERROR: size of order doesnt match original library")

In [ ]:
trimmed_seqs_df=pd.DataFrame(aep_seq, index=['seq']).transpose()
trimmed_seqs_df = trimmed_seqs_df.reset_index()
trimmed_seqs_df.columns=['aep','trimmed_seq']
trimmed_seqs_df.head()

In [ ]:
result_df=result_df.merge(trimmed_seqs_df,on='aep')

In [ ]:
result_df.head()

In [ ]:
# restriction size filtering:
bbs1="GAAGAC"
bsmb1="CGTCTC"
bsa1="GGTCTC"

In [ ]:
filtered=[]
for aep,seq in zip(result_df['aep'],result_df['seq']):
    if bsa1 in str(seq):
        filtered.append(aep)
        print("[+] Found BSA1 Restriction site: {} in {}".format(bsa1,aep))
        idx=result_df[result_df['aep'] == aep].index.to_list()
        if len(idx) > 1:
            raise Exception("[-] Multiple rows for one target")
        else:
            idx=idx[0]
            print("[+] Removing row {} from dataframe".format(idx))
            result_df=result_df.drop(idx)
    if bbs1 in str(seq):
        print("\t[+] Found BBS1 Restriction site: {} in {}".format(bbs1,aep))
    if bsmb1 in str(seq):
        print("\t[+] Found BSMB1 Restriction site: {} in {}".format(bsmb1,aep))
print("[-] Number of promotors with restriction sites: {}".format(len(filtered)))

In [ ]:
data_dir_excel=data_dir + "target_promotors.xlsx"
result_df.to_excel(data_dir_excel)

In [ ]:
additional_promotor_seqs=["AEP_03486","AEP_03466","AEP_03465","AEP_03459","AEP_02900"]